In [ ]:
%matplotlib inline
import os
from query.datasets.prelude import *
from pprint import pprint

In [ ]:
from django.db import models
faces = Face.objects.annotate(height=F('bbox_y2')-F('bbox_y1')).filter(frame__video__id=791, labeler__name='mtcnn', height__gte=0.3, track=OuterRef('pk')).annotate(min_frame=Min('frame__number'), max_frame=Max('frame__number'))
print(FaceTrack.objects.annotate(min_frame=Subquery(faces.values('min_frame'), output_field=models.IntegerField()), max_frame=Subquery(faces.values('max_frame'), output_field=models.IntegerField()))).query

In [ ]:
print(FaceTrack.objects.all().values('id').query)
print(FaceTrack.objects.all().filter(face__frame__video__channel='CNN').values('id').query)
print(FaceTrack.objects.annotate(height=F('face__bbox_y2')-F('face__bbox_y1')).values('id').query)

In [ ]:
print(FaceTrack.objects.all().count())
print(FaceTrack.objects.all().filter(face__frame__video__channel='CNN').count())
print(FaceTrack.objects.annotate(height=F('face__bbox_y2')-F('face__bbox_y1')).count())

In [ ]:
# Get total number of detected female vs male faces
gender_frames = Gender.objects.annotate(count=Count('tvnews_face__frame')).values('name', 'count')
total = gender_frames[0]['count'] + gender_frames[1]['count']
print('Female: {:.1f}%, Male: {:.1f}%'.format(gender_frames[1]['count'] / float(total) * 100, gender_frames[0]['count'] / float(total) * 100))
pd.DataFrame(list(gender_frames))

In [ ]:
# Get # of frames with a man vs. woman on them
shows = [t['show'] for t in Video.objects.distinct('show').values('show')]
counts = {}
for show in shows:
    print 'Computing for {}'.format(show)
    counts[show] = {}
    for gender in ['male', 'female']:
        counts[show][gender] = Frame.objects.filter(
            video__show=show, faceinstance__gender__name=gender).distinct('id').count()
    counts[show]['total'] = Frame.objects.filter(video__show=show).annotate(c=Count('faceinstance')).filter(c__gt=0).count()

print ''
    
for show, count in counts.iteritems():
    print('{}: female {:.1f}%, male {:.1f}%'.format(show, (count['female'] / float(count['total']) * 100), (count['male'] / float(count['total']) * 100)))
pd.DataFrame(counts)

In [ ]:
# Get # of total faces per show that are man vs. woman
def talking_heads(qs):
    return qs.annotate(height=F('bbox_y2')-F('bbox_y1')).filter(height__gte=0.25)

shows = [t['show'] for t in Video.objects.distinct('show').values('show')]
counts = {}
for show in shows:
    counts[show] = {}
    for gender in ['male', 'female']:
        counts[show][gender] = talking_heads(FaceInstance.objects.filter(frame__video__show=show, gender__name=gender)).count()
    counts[show]['total'] = talking_heads(FaceInstance.objects.filter(frame__video__show=show)).count()    

print ''
    
for show, count in counts.iteritems():
    print('{}: female {:.1f}%, male {:.1f}%'.format(show, (count['female'] / float(count['total']) * 100), (count['male'] / float(count['total']) * 100)))
    
pd.DataFrame(counts)

In [ ]:
# Get # of total faces per show that are man vs. woman
def talking_heads(qs):
    return qs.annotate(height=F('bbox_y2')-F('bbox_y1')).filter(height__gte=0.25)

id = 4457280
FaceFeatures.dropTempFeatureModel()
FaceFeatures.getTempFeatureModel([id])

shows = ['CNN Newsroom With Poppy Harlow']
counts = {}
for show in shows:
    counts[show] = {}
    for gender in ['male', 'female']:
        counts[show][gender] = talking_heads(FaceInstance.objects.filter(frame__video__show=show, gender__name=gender, facefeaturestemp__distto_4457280__gte=1.7)).count()
    counts[show]['total'] = talking_heads(FaceInstance.objects.filter(frame__video__show=show, facefeaturestemp__distto_4457280__gte=1.7)).count()    

print ''
    
for show, count in counts.iteritems():
    print('{}: female {:.1f}%, male {:.1f}%'.format(show, (count['female'] / float(count['total']) * 100), (count['male'] / float(count['total']) * 100)))
    
fropd.DataFrame(counts)

In [ ]:
from collections import defaultdict
labeler_names = [l['labeler__name'] for l in Face.objects.values('labeler__name').distinct()]
print(labeler_names)

videos = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
for frame in Frame.objects.filter(Q(video__show='Situation Room With Wolf Blitzer') | Q(video__show='Special Report With Bret Baier')).select_related('video')[:10000:10]:
    faces = Face.objects.filter(frame=frame).select_related('labeler')
    for face in faces:
        videos[frame.video.id][frame.id][face.labeler.name].append(face)
        
print(dict(videos).keys())
        
AREA_THRESHOLD = 0.02
DIST_THRESHOLD = 0.10

mistakes = defaultdict(lambda: defaultdict(tuple))
for video, frames in videos.iteritems():
    for frame, labelers in frames.iteritems():
        for labeler, faces in labelers.iteritems():
            for face in faces:
                if bbox_area(face) < AREA_THRESHOLD:
                    continue

                mistake = True
                for other_labeler in labeler_names:
                    if labeler == other_labeler: continue
                    other_faces = labelers[other_labeler] if other_labeler in labelers else []
                    for other_face in other_faces:
                        if bbox_dist(face, other_face) < DIST_THRESHOLD:
                            mistake = False
                            break

                    if mistake and len(other_faces) > 0:
                        mistakes[video][frame] = (faces, other_faces)
                        break
                else:
                    continue
                break

result = []
for video, frames in list(mistakes.iteritems())[:100]:
    for frame, (faces, other_faces) in frames.iteritems():
        result.append({
            'video': video,
            'start_frame': frame,
            'bboxes': [bbox_to_dict(f) for f in faces + other_faces]
        })
print(len(result))        